In [1]:
import requests
import time
import pandas as pd

In [2]:
base_url = "http://localhost:8000" 

In [3]:
def post_tweet_for_prediction(tweet, user):
    response = requests.post(f'{base_url}/predict', json={"text": tweet, "user": user})
    return response.json()

## user posting tweet

In [4]:
def store_posted_tweet(tweet_id, retweet_id, user_id, text, likes, retweets, safety_status):
    data = {
        "tweet_id": tweet_id,
        "retweet_id": retweet_id,
        "user_id": user_id,
        "text": text,
        "likes": likes,
        "retweets": retweets,
        "safety_status": safety_status
    }
    response = requests.post(f'{base_url}/store_tweet', json=data)
    return response.json()

In [5]:
def simulate_user_submission():
    tweets = [
        {"text": "This is an example tweet for testing purposes.", "user": "example_user"},
        {"text": "Another test tweet for batch prediction.", "user": "example_user"},
        {"text": "Yet another example of a tweet.", "user": "example_user"}
    ]

    for tweet in tweets:
        prediction = post_tweet_for_prediction(tweet["text"], tweet["user"])
        print("Prediction Result:", prediction)
        
        if prediction["logreg_result"] > 0:
            store_response = store_posted_tweet(
                tweet_id=prediction["tweet_id"],
                retweet_id=None,
                user_id=tweet["user"],
                text=prediction["tweet"],
                likes=prediction["likes"],
                retweets=prediction["retweets"],
                safety_status=None  # Initial safety status is None
            )
            print("Store Tweet Response:", store_response)

In [6]:
simulate_user_submission()

Prediction Result: {'tweet_id': '181512ed', 'tweet': 'This is an example tweet for testing purposes.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.01582927467824479, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None}
Prediction Result: {'tweet_id': 'c856facb', 'tweet': 'Another test tweet for batch prediction.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.5926823273721858, 'logreg_result': 1, 'cnn_prob': None, 'cnn_result': None}
Store Tweet Response: {'tweet_id': 'c856facb', 'text': 'Another test tweet for batch prediction.', 'retweet_id': None, 'retweets': 0, 'id': 2, 'user_id': 'example_user', 'likes': 0, 'safety_status': None}
Prediction Result: {'tweet_id': 'db8a5f99', 'tweet': 'Yet another example of a tweet.', 'user': 'example_user', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.340711603105755, 'logreg_result': 0, 'cnn_prob': None, 'cnn_result': None}


In [7]:
time.sleep(40)  # Wait to ensure the scheduler runs at least once

## admin viewing tweets

In [8]:
# Function to fetch all tweets
def fetch_all_tweets():
    response = requests.get(f'{base_url}/fetch_tweets')
    return response.json()

In [11]:
# Fetch and display tweets after batch prediction
tweets_after_prediction = fetch_all_tweets()

{'tweet_id': '2582afe1', 'retweet_id': None, 'tweet': 'well that escalated quickly mtb motor woowooelectricsta', 'user': 'test_user7', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.37973127429662035, 'logreg_result': 0, 'cnn_prob': 0.024221671745181084, 'cnn_result': 0, 'admin_result': None}


In [12]:
for i in range(len(tweets_after_prediction)):
    print(tweets_after_prediction[i])

{'tweet_id': '2582afe1', 'retweet_id': None, 'tweet': 'well that escalated quickly mtb motor woowooelectricsta', 'user': 'test_user7', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.37973127429662035, 'logreg_result': 0, 'cnn_prob': 0.024221671745181084, 'cnn_result': 0, 'admin_result': None}
{'tweet_id': 'ea3ec30d', 'retweet_id': None, 'tweet': 'i am trying to get one i have emailed channel asking them for a source that would be acceptable by wikipedia s standards hopefully they will reply in the next working day or two', 'user': 'user0', 'likes': 0, 'retweets': 0, 'logreg_prob': 0.060409772267669935, 'logreg_result': 0, 'cnn_prob': 0.021217793226242065, 'cnn_result': 0, 'admin_result': None}
{'tweet_id': '6251ca46', 'retweet_id': None, 'tweet': 'congratulations congratulations floquenbeam you have won the asshole of the year award you have won this award by saying mean things about and to other people in order to claim your award you must call asshole once again congratulations on your 

In [14]:
# Print tweets where logreg_result does not match cnn_result
for tweet in tweets_after_prediction:
    if tweet['logreg_result'] != tweet['cnn_result']:
        print(f"Discrepancy found for tweet_id {tweet['tweet_id']}:")
        print(f"  Logistic Regression Result: {tweet['logreg_result']}")
        print(f"  CNN Result: {tweet['cnn_result']}")
        print(f"  Tweet: {tweet['tweet']}\n")

Discrepancy found for tweet_id c856facb:
  Logistic Regression Result: 1
  CNN Result: 0
  Tweet: Another test tweet for batch prediction.



## user reporting tweets

In [ ]:
import requests

# Example tweet ID to report and the safety status (e.g., 1 for hate speech)
report_data = {
    "tweet_id": "a975f406",
    "safety_status": 1  # Assume 1 indicates hate speech
}

# Function to report a tweet
def report_tweet(report_data):
    url = f"{base_url}/report_tweet"
    response = requests.post(url, json=report_data)
    
    if response.status_code == 200:
        print("Report submitted successfully.")
    else:
        print(f"Failed to submit report: {response.status_code}")
        print(f"Response: {response.json()}")

# Run the simulation
report_tweet(report_data)
